# Bibliotecas

In [1]:
import ranking
import salvar_resultados as sr

import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import funcoes_bert as fb

import time

## Constantes

In [2]:
SEED = 42

# Lendo os Arquivos

In [3]:
arquivos = ['ale_1_1']
#arquivos = ['ale_5_1']
#arquivos = ['hn_1_1']
#arquivos = ['hn_5_1']

lista_df_treino = []
lista_df_val = []
lista_df_teste = []
for arquivo in arquivos:

    df = pd.read_csv(f"Dados/Datasets/Treino-Validação/Treino/{arquivo}_treino.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df_treino.append(df)

    df = pd.read_csv(f"Dados/Datasets/Treino-Validação/Validação/{arquivo}_validação.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df_val.append(df)

    df = pd.read_csv(f"Dados/Datasets/Teste/{arquivo}_teste.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df_teste.append(df)

In [4]:
#lista_df_val[0].dtypes

In [5]:
#print(f"Tamanho do Dataset de Treino 1 por 1:\n\n\t| Ale\t| Hard\t|\n1 por 1\t| {lista_df_treino[0].shape[0]}\t| {lista_df_treino[2].shape[0]}\t|\n5 por 1\t| {lista_df_treino[1].shape[0]}\t| {lista_df_treino[3].shape[0]}\t|")
#print(f"\nTamanho do Dataset de Validação 1 por 1:\n\n\t| Ale\t| Hard\t|\n1 por 1\t| {lista_df_val[0].shape[0]}\t| {lista_df_val[2].shape[0]}\t|\n5 por 1\t| {lista_df_val[1].shape[0]}\t| {lista_df_val[3].shape[0]}\t|")
#print(f"\nTamanho do Dataset de Teste 1 por 1:\n\n\t| Ale\t| Hard\t|\n1 por 1\t| {lista_df_teste[0].shape[0]}\t| {lista_df_teste[2].shape[0]}\t|\n5 por 1\t| {lista_df_teste[1].shape[0]}\t| {lista_df_teste[3].shape[0]}\t|")


In [6]:
from transformers import TFBertForSequenceClassification

# BERT

In [7]:
def salvar_modelo_bert(modelo, nome):
    
    nome_arquivo = f"Dados/Modelos/BERT/{nome}/"
    modelo.save_pretrained(nome_arquivo)

In [8]:
def criar_dicionario(indice, titulo, ean, categoria, colunas):
    
    return {
            colunas[0] : indice, colunas[1] : titulo, colunas[2] : ean, colunas[3] : categoria
           }


def criar_df_match(df_concat, ean_repetido, coluna_saida = 'titulo_sa'):

    COLUNAS = ("indice", coluna_saida, "ean", "categoria")

    df_matches = pd.DataFrame(columns = COLUNAS)
    for ean in ean_repetido:

        # pega o indice da primeira linha com aquele EAN
        filtro = (df_concat['ean'] == ean)
        indice = next(iter(filtro.index[filtro]))

        dicionario = criar_dicionario(
                                    indice = indice,
                                    titulo = df_concat.loc[indice][coluna_saida],
                                    ean = df_concat.loc[indice]['ean'],
                                    categoria = df_concat.loc[indice]['categoria'],
                                    colunas = COLUNAS
                                    )

        df_matches = df_matches.append(dicionario, ignore_index = True)

    df_matches.sort_values('indice', inplace = True)
    df_matches.reset_index(drop = True, inplace = True)

    return df_matches


## Rodando o BERT

In [9]:
dir_metricas = "Dados/Resultados/Ranqueado/BERT"

In [10]:
lista_df_resultado = []
tam = 51
for nome, df_treino, df_val, df_teste in zip(arquivos, lista_df_treino, lista_df_val, lista_df_teste):

    df_teste = df_teste[:tam]
    df_treino = df_teste[:tam]
    df_val = df_teste[:tam]
    
    df_concat, df_matches = ranking.criar_df_teste(df_teste, "titulo_1", "titulo_2", 'titulo')

    X_treino = df_treino[["titulo_1", "titulo_2"]]
    X_val = df_val[["titulo_1", "titulo_2"]]
    X_teste = df_concat["titulo"]

    y_treino = df_treino["match"].to_list()
    y_val = df_val["match"].to_list()
    #y_teste = df_teste["match"].to_list()

    inicio_tempo = time.time()

    #(modelo, historico, embedding) = fb.pipeline_bert(X_treino, y_treino, X_val, y_val, X_teste)
    (modelo, embedding) = fb.pipeline_bert(X_teste)
    resultado = ranking.calcular_dis_2_vetores_cond(embedding, df_matches)
    
    final_tempo = time.time()
    
    # adicionando o tempo que demorou para o algoritmo rodar
    ranking.calcular_tempo(df_matches, inicio_tempo, final_tempo)

    # calcular acuracia_k e match rank
    indices, valores = ranking.calcular_metricas(df_matches, df_concat, resultado)

    #pd.DataFrame.from_dict(historico.history).to_csv(f'Dados/Resultados/Ranqueado/BERT/Histórico/{nome}_historico.csv', index = False)

    sr.salvar_resultado(nome, df_matches, dir_metricas)
    sr.salvar_resultado_categoria(nome, df_matches, dir_metricas)

    #salvar_modelo_bert(modelo, nome)

    break

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
c:\Users\llvs2\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [11]:
#print(classification_report(y_teste, y_pred))

In [12]:
    a

NameError: name 'a' is not defined

In [ ]:
'''print(classification_report(y_test, y_pred))

print(confusion_matrix(y_test, y_pred))'''

In [ ]:
#lista_df_resultado

# Juntando os Relatórios

### Carregando e Concatenando

Lê os arquivos com os resultados e depois concatena em um único dataframe

In [ ]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']
lista_df_resultados = []
for nome in arquivos:

    df = pd.read_csv(f"Dados/Resultados/BERT/Relatório/{nome}_relatório.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df_resultados.append(df)

df_resultados = pd.concat(lista_df_resultados, ignore_index = False)

In [ ]:
df_resultados.rename(columns = {'Unnamed: 0':'info'}, inplace = True)
df_resultados

### Salvando

Salvando todos os resultados em um único arquivo

In [ ]:
df_resultados.to_csv(f'Dados/Resultados/BERT/relatório_completo.csv', index = False)

# Salvando os Resultados por Categoria

## Funções

In [ ]:
def salvar_por_categoria(df_r, categoria, nome):

    df_r[df_r["categoria"] == categoria].to_csv(f'Dados/Resultados/BERT/Resultado/{categoria}/{nome}_y.csv', index = False)


def relatorio_por_categoria(df_r, categoria, nome):

    y_teste = df_r[df_r["categoria"] == categoria]['match']
    y_pred = df_r[df_r["categoria"] == categoria]['pred']

    relatorio = classification_report(y_teste, y_pred, output_dict = True)
    df_relatorio = pd.DataFrame(relatorio).transpose()
    df_relatorio['modelo'] = nome

    df_relatorio.to_csv(f'Dados/Resultados/BERT/Relatório/{categoria}/{nome}_relatório.csv', index = True)

## Leitura e Salvamento

In [ ]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']
lista_df_resultados = []
for nome in arquivos:

    df_r = pd.read_csv(f"Dados/Resultados/BERT/Resultado/{nome}_y.csv")
    lista_df_resultados.append(df_r)

In [ ]:
lista_categorias = ("celulares", "notebooks", "geladeiras", "fogoes", "tvs")
for nome, df_r in zip(arquivos, lista_df_resultados):

    for categoria in lista_categorias:

        salvar_por_categoria(df_r, categoria, nome)
        relatorio_por_categoria(df_r, categoria, nome)